In [1]:
# Sentiment Analysis Dashboard with Kaggle Dataset (Google Colab Ready with Google Drive and Pyngrok)

# 1. Install Requirements
!pip install pandas textblob plotly dash pyngrok vaderSentiment --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 13.8 MB/s eta 0:00:00


In [2]:
# 2. Import Libraries
import pandas as pd
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
from pyngrok import ngrok
from google.colab import drive
import threading

In [3]:
# 3. Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 4. Load datasets/DatasetUAS from Google Drive
# Replace with your file paths in Google Drive
humint = pd.read_csv('/content/drive/MyDrive/datasets/DatasetUAS/Reviews.csv')
humint = humint[['Score', 'Text']].dropna()

In [5]:
osint = pd.read_csv('/content/drive/MyDrive/datasets/DatasetUAS/training.1600000.processed.noemoticon.csv', encoding='latin-1', names=['polarity', 'id', 'date', 'query', 'user', 'text'])
osint = osint[['text']]

In [6]:
sigint = pd.read_csv('/content/drive/MyDrive/datasets/DatasetUAS/spam.csv', encoding_errors='ignore')
sigint = sigint[['v2']]

In [7]:
# 5. Sentiment Analysis Function with VADER
analyzer = SentimentIntensityAnalyzer()

def classify_sentiment(text):
    score = analyzer.polarity_scores(str(text))['compound']
    if score >= 0.05:
        return 'Positive'
    elif score <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

In [8]:
# Apply sentiment analysis
humint['Sentiment'] = humint['Text'].apply(classify_sentiment)

In [9]:
osint['Sentiment'] = osint['text'].apply(classify_sentiment)

In [10]:
sigint['Sentiment'] = sigint['v2'].apply(classify_sentiment)

In [11]:
# 6. Summarize Data
def summarize_data(df, source):
    summary = df['Sentiment'].value_counts().reset_index()
    summary.columns = ['Sentiment', 'Count']
    summary['Source'] = source
    return summary

hum_summary = summarize_data(humint, 'HUMINT')
osint_summary = summarize_data(osint, 'OSINT')
sigint_summary = summarize_data(sigint, 'SIGINT')

summary_df = pd.concat([hum_summary, osint_summary, sigint_summary])

In [12]:
# 7. Build Dashboard with Dash
app = Dash(__name__)

app.layout = html.Div([
    html.H1('Sentiment Analysis Dashboard'),
    dcc.Dropdown(
        id='source-dropdown',
        options=[
            {'label': 'HUMINT', 'value': 'HUMINT'},
            {'label': 'OSINT', 'value': 'OSINT'},
            {'label': 'SIGINT', 'value': 'SIGINT'}
        ],
        value='HUMINT'
    ),
    dcc.Graph(id='sentiment-bar')
])

@app.callback(
    Output('sentiment-bar', 'figure'),
    Input('source-dropdown', 'value')
)
def update_graph(selected_source):
    filtered_df = summary_df[summary_df['Source'] == selected_source]
    fig = px.bar(filtered_df, x='Sentiment', y='Count', color='Sentiment',
                 title=f'Sentiment Distribution for {selected_source}')
    return fig

In [14]:
# 8. Run Dashboard with Pyngrok
# Replace this with your actual ngrok token
ngrok.set_auth_token('NGROK_API_KEY')

port = 8050
public_url = ngrok.connect(port)
print(f'Dashboard running at: {public_url}')

# Run Dash app in separate thread
def run_app():
    app.run(host='0.0.0.0', port=port)

thread = threading.Thread(target=run_app)
thread.start()

# Display the public URL
import time
print("Waiting for the dashboard to start...")
time.sleep(5)
print(f'Access your dashboard here: {public_url}')

Dashboard running at: NgrokTunnel: "https://c6e7-34-16-174-202.ngrok-free.app" -> "http://localhost:8050"
Waiting for the dashboard to start...


<IPython.core.display.Javascript object>

Access your dashboard here: NgrokTunnel: "https://c6e7-34-16-174-202.ngrok-free.app" -> "http://localhost:8050"
